In [ ]:
from tempo_ql import GenericDataset, formats, QueryEngine, FileVariableStore
import duckdb
import numpy as np
from pathlib import Path
import pandas as pd

This example will use the MIMIC-IV-OMOP dataset available [here](https://physionet.org/content/mimic-iv-demo-omop/0.9/). Please download it and set the path to it below.

In [ ]:
base_path = 'tempo_ql/omop/mimic-iv-demo-data-in-the-omop-common-data-model-0.9/1_omop_data_csv'

In [ ]:
# Add data from the MIMIC-IV OMOP dataset

local_db = duckdb.connect(":memory:myconn")
for csv_file in Path(base_path).glob('*.csv'):
    table_name = csv_file.stem
    local_db.execute(f"create table {table_name} as select * from read_csv_auto('{csv_file}', header=true, ignore_errors=true, parallel=false)")

# Initialize query engine and variable store
var_store = FileVariableStore('example_data')
query_engine = QueryEngine(GenericDataset("duckdb:///:memory:myconn", formats.omop(id_field='person_id')), 
                           variable_stores=[var_store])

In [ ]:
query_engine.dataset.get_scopes()

In [ ]:
# get available concepts
names = query_engine.dataset.list_data_elements(scope='Person', return_counts=True)
names

In [ ]:
# Get all visit occurrence IDs
query_engine.get_ids()

In [ ]:
# perform one-off queries
visits = query_engine.query("{name contains /blood pressure/i; scope = Measurement}")
visits

In [ ]:
# Run future queries only within a random subset if desired
random_sample = np.random.choice(query_engine.get_ids(), size=50)
query_engine.dataset.set_trajectory_ids(random_sample)

In [ ]:
# we can now store variables by name...
var_store['SimpleTemperature'] = query_engine.query("union({Temperature Celsius}, ({Temperature Fahrenheit} - 32) * 5 / 9) where #value < 50")

In [ ]:
# ... and then use their results later
query_engine.query("last SimpleTemperature before #now every 24 hours", return_subqueries=True)

In [ ]:
# Start an interactive widget
query_engine.interactive() # api_key=...

In [ ]:
# Work in a JSON file to save queries (e.g. for version control)
query_engine.interactive(file_path='example_data/test_queries.json') # api_key=...

In [ ]:
# Later, query from your saved file
query_engine.query_from('example_data/test_queries.json')